# Lesson 12. Variance reduction &mdash; common random numbers

### SA421 Fall 2015

## Warm up

**Problem.** Let $X$ be a continuous uniform random variable on $[a, b]$.
Recall that the cdf of $X$ is

$$F_X(x) = \begin{cases}
    0 & \text{if } x < a\\
    (x-a) / (b-a) & \text{if } a \le x \le b\\
    1 & \text{if } x > b
\end{cases}$$

In the code cell below, the function `uniform_SA421`, which is supposed to generate random variates of $X$, is partially defined.
Use the inverse transform method to complete the definition of `uniform_SA421`.

In [ ]:
# Import NumPy's random number generator
from numpy.random import rand

# uniform_SA421(a, b, verbose)
# Generates a random variate from the Uniform[a, b] distribution
# Prints out the random number used if verbose == True
def uniform_SA421(a, b, verbose):
    # Generate random number
    u = 
    
    # Print random number used, if verbose == True
    if verbose == True:
        print("  Random number used = {0}".format(u))
    
    # Compute random variate using inverse cdf
    x = 
    
    # Return random variate
    return x

Test your definition of `uniform` below:

In [ ]:
uniform_SA421(100, 200, True)

## Motivation

* Consider the Fantastic Dan problem once again: single server with a single queue.


* Customer interarrival times are uniformly distributed between 10 and 25 minutes.


* **System 1.**  In the current system, service times are uniformly distributed between 30 and 45 minutes.


* **System 2.** Fantastic Dan is considering a new haircutting technique. In this hypothetical system, service times are uniformly distributed between 15 and 20 minutes. 


* Fantastic Dan wants to know what effect this new haircutting technique would have on the time average number of customers waiting to get a haircut (in the queue).

* In the cell below is SimPy code to simulate Fantastic Dan's shop for 1 hour. This code is nearly identical to what we've used before. Some notes:
    * The code is set up to print the arrival times of customers.
    * The parameters `P.serviceTimeMin` and `P.serviceTimeMax` have been initialized to `None` &mdash; we'll fill those in later.
    * The `model` function is set up to take a seed as input. 

In [ ]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed-setting function from NumPy
from numpy.random import seed

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with uniformly distributed
    # interarrival times between 10 and 25 minutes
    interarrivalTimeMin = 10
    interarrivalTimeMax = 25
    
    # Service times are uniformly distributed, but min/max values will vary
    serviceTimeMin = None
    serviceTimeMax = None
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Simulate for 2 hours
    simulationTimeMax = 2 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        # self.name is the name we give this customer when we generate him/her
        #   in the Entrance process
        print("Time {1}: {0} arrives and joins queue".format(self.name, now())) 
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        serviceTime = uniform_SA421(P.serviceTimeMin, P.serviceTimeMax, False)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = uniform_SA421(P.interarrivalTimeMin, P.interarrivalTimeMax, False)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name = "Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model(inputSeed):    
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Let's run the simulation once for System 1:

In [ ]:
##### Experiment - System 1 #####
# Service time parameters for System 1
P.serviceTimeMin = 
P.serviceTimeMax = 

# Run simulation with seed 123
model(123)

* Let's also run the simulation once for System 2:

In [ ]:
##### Experiment - System 2 #####
# Service time parameters for System 2
P.serviceTimeMin = 
P.serviceTimeMax = 

# Run simulation with seed 123
model(123)

* The customer arrival times are different in the two systems, even though 
    - the interarrival time distribution is the same between the two systems,
    - we used the same seed in both simulations.
    

* Intuitively, we don't want this to happen: we want our analysis to focus on the difference between the two systems, which is the <span style="color:#a00000">service time</span> distribution, not the interarrival time distribution.


* Why does this difference occur?

## The effect of using a single stream of random numbers

* Remember that random variates are generated using a sequence (or stream) of random numbers.


* In the above simulations, both interarrival times and service times are being sampled using <span style="color:#a00000">a single stream of random numbers</span>.


* As a result, if the service times are different between the two systems, the interarrival times will eventually be generated using different random numbers.


* To illustrate this, here is the same SimPy code as above, except every time we generate a random variate:
    - We print out the corresponding random number.
    - We also print out what type of variate we've generated (interarrival or service time).

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with uniformly distributed
    # interarrival times between 10 and 25 minutes
    interarrivalTimeMin = 10
    interarrivalTimeMax = 30
    
    # Service times are uniformly distributed, but min/max values will vary
    serviceTimeMin = None
    serviceTimeMax = None
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Simulate for 2 hours
    simulationTimeMax = 2 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        yield request, self, R.server   
        
        # Customer is released from queue and starts service
        # The end = "" keyword argument for the print function prevents print
        #   from starting a new line
        print("Service time generated for {0}:".format(self.name), end = "")
        serviceTime = uniform_SA421(P.serviceTimeMin, P.serviceTimeMax, True)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Create name of customer
            customerName = "Customer {0}".format(nCustomers)
            
            # Wait until the next arrival
            print("Interarrival time generated for {0}:".format(customerName), end = "")
            interarrivalTime = uniform_SA421(P.interarrivalTimeMin, P.interarrivalTimeMax, True)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name = customerName)
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model(inputSeed):    
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers, monitored = True)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Let's run the simulation for System 1:

In [ ]:
##### Experiment - System 1 #####
# Service time parameters for System 1
P.serviceTimeMin = 30
P.serviceTimeMax = 45

# Run simulation with seed 123
model(123)

* And now, let's run the simulation for System 2:

In [ ]:
##### Experiment - System 2 #####
# Service time parameters for System 2
P.serviceTimeMin = 15
P.serviceTimeMax = 20

# Run simulation with seed 123
model(123)

* As we can see, the same stream of random numbers is used for both interarrival times and service times.


* Because the service times in the two systems follow different distributions, eventually, each seed is used to generate different quantities in each system.
    - For example, `0.9807641983846155` is the random number used for the interarrival time of Customer 4 in System 1 and the service time of Customer 2 in System 2.

## Using separate streams and common random numbers

*  We want our analysis to focus on the difference between the two systems: the <span style="color:#a00000">service time</span> distribution, not the interarrival time distribution.


* This in turn will help us <span style="color:#a00000">reduce the variance</span> in our estimates of the difference between the two systems.


* By setting up <span style="color:#a00000">separate streams</span> for interarrival times and service times, we can ensure that the interarrival times are the same across both systems.


* This is called the **common random number** technique: ensuring that common random numbers are used to generate the same random variates for <span style="color:#a00000">matching parts</span> of alternate systems.
    - Other names: **correlated sampling**, **matched streams**, **matched pairs**

## Separate streams in NumPy: RandomState

* Let's implement separate streams in the Fantastic Dan simulation.


* First, we need to decide how to set up our random number streams.
    - In this case, let's set up 1 stream for interarrival times and 1 stream for service times.


* **Rule of thumb.** Set up a separate stream for each random quantity in your simulation.
    

* To set up separate streams, we can use `RandomState()`. [Here is the documentation.](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.RandomState.html)

In [ ]:
##### Additional Setup #####
# Import RandomState from NumPy for separate streams
from numpy.random import RandomState

* Like we do for parameters, resources, and monitors, we'll place all our streams into one class, called `S`, so that streams all uniformly start with the prefix `S.`
    - For example, `S.inter` for the stream of interarrival times.


* To sample from distributions, we need to <span style="color:#a00000;">prepend</span> the name of the stream before the sampling function name.
    - For example, <span style="color:#a00000; font-family:monospace">S.inter.</span>`uniform(low = ..., high = ...)`.
    

* Similarly, to provide a seed for these streams, we <span style="color:#a00000;">prepend</span> the `seed` function with the name of the stream.
    - For example, <span style="color:#a00000; font-family:monospace">S.inter.</span>`seed(123)`

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with uniformly distributed
    # interarrival times between 10 and 25 minutes
    interarrivalTimeMin = 10
    interarrivalTimeMax = 25
    
    # Service times are uniformly distributed, but min/max values will vary
    serviceTimeMin = None
    serviceTimeMax = None
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Simulate for 2 hours
    simulationTimeMax = 2 * 60
    
    
##### Streams #####
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now())) 
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        serviceTime = uniform_SA421(P.serviceTimeMin, P.serviceTimeMax, False)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = uniform_SA421(P.interarrivalTimeMin, P.interarrivalTimeMax, False)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name = "Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model(inputSeed):    
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers, monitored = True)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Now, let's look again at the customer arrival times in System 1 for one replication:

In [ ]:
##### Experiment - System 1 #####
# Service time parameters for System 1
P.serviceTimeMin = 30
P.serviceTimeMax = 45

# Run model once
model(123, 123)

* Similarly, for System 2:

In [ ]:
##### Experiment - System 2 #####
# Service time parameters for System 2
P.serviceTimeMin = 15
P.serviceTimeMax = 20

# Run model once
model(123, 123)

* As we can see, the arrival times in both systems now match!

## If we have time &mdash; with a neighbor

**Problem.** Instead of using the new haircutting technology, Fantastic Dan has decided to team up with Bartender Becky to add a bar service to the shop so customers can relax after their haircuts.

Fantastic Dan can be pretty aggressive with the scissors, so every customer has a drink with Bartender Becky after his or her haircut.
Bartender Becky can serve only 1 customer at a time, so a queue will form at the bar if Becky is busy.
The time it takes for Bartender Becky to serve a customer is exponentially distributed with a mean of 5 minutes.

Write SimPy code to simulate this new system for 8 hours. <span style="color:#a00000">Use separate streams of random numbers to generate interarrival times, haircut times, and bar times.</span> Run your simulation once, and compute the time average number of customers waiting for Bartender Becky. Does your answer make sense?